<a href="https://colab.research.google.com/github/ITServices-AIOps/SandBox/blob/main/Backlog_Analytical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Celda NUEVA (o al inicio, después de montar Drive): Instalar gspread
!pip install gspread google-auth oauth2client

In [ ]:
# -*- coding: utf-8 -*-
"""
Notebook para Análisis Consolidado de Tareas de TI.
"""

# Celda 1: Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Celda 2: Importar Bibliotecas
import pandas as pd
import numpy as np
import os # Para manejar rutas de archivo

print("Bibliotecas importadas y Drive montado.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Bibliotecas importadas y Drive montado.


In [ ]:
# Celda 3: Definir Rutas (¡AJUSTAR SEGÚN TU DRIVE!)
# Ejemplo: Si tus archivos están en "Mi unidad/DatosManageEngine/"
ruta_base = "/content/drive/MyDrive/BacklogProcesar"
archivo_ordenes = os.path.join(ruta_base, "Ordenes.gsheet")
archivo_incidentes = os.path.join(ruta_base, "Incidentes.gsheet")
archivo_cambios = os.path.join(ruta_base, "Cambios.gsheet")


In [ ]:
# --- REEMPLAZO para la Celda 4: Leer directamente desde Google Sheets ---
import gspread
from google.colab import auth
import pandas as pd
import os # Sigue siendo útil para nombres base, etc.

# Autenticar al usuario (esto abrirá una ventana de autenticación de Google)
auth.authenticate_user()

# Autorizar gspread para acceder a las hojas
import google.auth
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

print("Autenticación y autorización completadas.")

# --- ¡ACCIÓN REQUERIDA! ---
# Escribe los NOMBRES EXACTOS de tus archivos Google Sheet
# (tal como aparecen en Google Drive, sin la extensión .gsheet)
nombres_google_sheets = {
    'Ordenes': "Ordenes",       # El nombre de tu archivo de Órdenes
    'Incidentes': "Incidentes",   # El nombre de tu archivo de Incidentes
    'Cambios': "Cambios"       # El nombre de tu archivo de Cambios
}
# Opcionalmente, si están en una carpeta específica, puedes añadir la ruta
# o usar la URL completa de cada hoja si los nombres no son únicos.

dfs = {} # Diccionario para guardar los DataFrames

for nombre_proceso, nombre_sheet in nombres_google_sheets.items():
    try:
        print(f"Intentando abrir Google Sheet: '{nombre_sheet}'...")
        # Abrir la hoja de cálculo por su nombre
        worksheet = gc.open(nombre_sheet).sheet1 # Accede a la primera hoja (tab) por defecto.
                                                 # Usa .worksheet("NombreDeLaPestaña") si no es la primera.

        # Obtener todos los valores como una lista de listas
        data = worksheet.get_all_values() # O usa get_all_records() si la primera fila son headers claros

        # Convertir a DataFrame de Pandas
        if data:
            # Asume que la primera fila son los encabezados
            df_cargado = pd.DataFrame(data[1:], columns=data[0])
            print(f"Hoja '{nombre_sheet}' cargada con éxito en un DataFrame.")
            dfs[nombre_proceso] = df_cargado
            dfs[nombre_proceso]['Origen_Proceso'] = nombre_proceso # Añadir columna de origen
        else:
            print(f"Advertencia: La hoja '{nombre_sheet}' parece estar vacía.")
            dfs[nombre_proceso] = pd.DataFrame() # Crear DF vacío si no hay datos
            dfs[nombre_proceso]['Origen_Proceso'] = nombre_proceso


    except gspread.exceptions.SpreadsheetNotFound:
        print(f"ERROR: No se encontró una Google Sheet con el nombre: '{nombre_sheet}'. Verifica el nombre.")
    except Exception as e:
        print(f"ERROR inesperado al cargar la hoja '{nombre_sheet}': {e}")

# Verificar qué DataFrames se cargaron
print(f"\nDataFrames cargados desde Google Sheets: {list(dfs.keys())}")

# A partir de aquí, el resto del script (Paso 3 en adelante) debería funcionar
# con los DataFrames en el diccionario 'dfs'.

Autenticación y autorización completadas.
Intentando abrir Google Sheet: 'Ordenes'...
Hoja 'Ordenes' cargada con éxito en un DataFrame.
Intentando abrir Google Sheet: 'Incidentes'...
Hoja 'Incidentes' cargada con éxito en un DataFrame.
Intentando abrir Google Sheet: 'Cambios'...
Hoja 'Cambios' cargada con éxito en un DataFrame.

DataFrames cargados desde Google Sheets: ['Ordenes', 'Incidentes', 'Cambios']


In [ ]:
# Celda 5: Inspeccionar Columnas de Cada DataFrame
for nombre, df in dfs.items():
    print(f"\n--- Columnas en {nombre} ---")
    print(df.columns.tolist())
    print(f"\n--- Primeras filas de {nombre} ---")
    print(df.head(3))
    print(f"\n--- Información de {nombre} ---")
    df.info()
    print("-" * 50)

# --- ANÁLISIS VISUAL ---
# Revisa la salida de esta celda detenidamente.
# Identifica las columnas que contienen:
#   1. Un ID único (ID de la solicitud, ID_de_cambio, etc.)
#   2. La descripción principal de la tarea (Asunto, Titulo, Descripcion, Resolución, etc.)
#   3. El técnico asignado (Técnico, Administrador_de_cambios, etc.)
#   4. El grupo asignado (Grupo de soporte asignado, Implementador, etc.)
#   5. El tipo específico (Tipo de ticket, Tipodecambio, etc. - Opcional pero útil)
# Anota los nombres EXACTOS para el siguiente paso.


--- Columnas en Ordenes ---
['Cuenta', 'ID de la solicitud', 'Tipo de ticket', 'Tipo de Incidencia', 'Estado de solicitud', 'Prioridad', 'Asunto', 'Técnico', 'Grupo de soporte asignado', 'Categoría de servicio', 'Mes', 'Hora de creación', 'Hora de la última actualización', 'Hora de resolución', 'Solicitante', 'Resolución', 'Tiempo de resolución de SLA', 'Categoría', 'Categoría 2', 'Categoría 3', 'Producto', 'Imputable', 'Origen_Proceso']

--- Primeras filas de Ordenes ---
     Cuenta ID de la solicitud    Tipo de ticket Tipo de Incidencia  \
0  Procesar            2124443  Orden de Trabajo        No asignado   
1  Procesar            2121791  Orden de Trabajo        No asignado   
2  Procesar            2080167  Orden de Trabajo        No asignado   

  Estado de solicitud Prioridad  \
0             Cerrado      Baja   
1             Cerrado      Baja   
2             Cerrado      Baja   

                                              Asunto  \
0                      MDL | reinicio no

In [ ]:
# Celda 6: Definir Nombres Objetivo y Mapeos Específicos
# ¡ACCIÓN REQUERIDA! Ajusta los mapeos según los nombres reales de tus columnas.

nombres_objetivo = {
    'ID_Original': 'ID_Original', # El ID específico del ticket/cambio/orden
    'Descripcion_Tarea': 'Descripcion_Tarea', # La descripción unificada de la tarea
    'Tecnico': 'Tecnico', # Persona asignada
    'Grupo': 'Grupo', # Grupo asignado
    'Tipo_Ticket_Original': 'Tipo_Ticket_Original', # El tipo original si existe (e.g., 'Incidente Urgente', 'Cambio Normal')
    'Origen_Proceso': 'Origen_Proceso' # Ya la creamos: 'Incidentes', 'Cambios', 'Ordenes'
}

# --- EJEMPLO DE MAPEADOS (¡DEBES ADAPTARLOS!) ---
mapeo_ordenes = {
    'ID de la solicitud': 'ID_Original',       # Ajusta según tu columna de ID en Órdenes
    'Asunto': 'Descripcion_Tarea',         # Ajusta - Elige la mejor descripción
    # 'Descripcion': 'Descripcion_Secundaria', # Podrías guardar otra descripción si es útil
    'Técnico': 'Tecnico',                # Ajusta
    'Grupo de soporte asignado': 'Grupo', # Ajusta
    'Tipo de ticket': 'Tipo_Ticket_Original'   # Ajusta si existe esta columna
    # 'Origen_Proceso': 'Origen_Proceso' # Ya existe
}

mapeo_incidentes = {
    'ID de la solicitud': 'ID_Original',       # Ajusta
    'Asunto': 'Descripcion_Tarea',         # Ajusta - Elige la mejor descripción
    'Técnico': 'Tecnico',                # Ajusta
    'Grupo de soporte asignado': 'Grupo', # Ajusta
    'Tipo de ticket': 'Tipo_Ticket_Original'   # Ajusta si existe
    # 'Origen_Proceso': 'Origen_Proceso' # Ya existe
}

mapeo_cambios = {
    'ID_de_cambio': 'ID_Original',         # Ajusta
    'Titulo': 'Descripcion_Tarea',         # Ajusta - Elige la mejor descripción
    'Administrador_de_cambios': 'Tecnico', # Ajusta
    'Implementador': 'Grupo',             # Ajusta
    'Tipodecambio': 'Tipo_Ticket_Original'  # Ajusta
    # 'Origen_Proceso': 'Origen_Proceso' # Ya existe
}

# Diccionario de mapeos para iterar
mapeos_todos = {
    'Ordenes': mapeo_ordenes,
    'Incidentes': mapeo_incidentes,
    'Cambios': mapeo_cambios
}


In [ ]:

# Celda 7: Aplicar Selección y Renombrado
dfs_estandarizados = {}

for nombre, df in dfs.items():
    if nombre in mapeos_todos:
        mapeo_actual = mapeos_todos[nombre]
        columnas_originales_necesarias = list(mapeo_actual.keys())

        # Verificar si todas las columnas necesarias existen
        columnas_faltantes = [col for col in columnas_originales_necesarias if col not in df.columns]
        if columnas_faltantes:
            print(f"ADVERTENCIA en {nombre}: Faltan las columnas {columnas_faltantes}. Se omitirán del mapeo.")
            # Actualizar el mapeo para excluir las columnas faltantes
            mapeo_actual = {k: v for k, v in mapeo_actual.items() if k in df.columns}
            columnas_originales_necesarias = list(mapeo_actual.keys()) # Actualizar lista

        # Asegurarse de incluir 'Origen_Proceso'
        if 'Origen_Proceso' not in columnas_originales_necesarias:
             columnas_originales_necesarias.append('Origen_Proceso')

        # Seleccionar solo las columnas que existen y son necesarias
        df_seleccionado = df[columnas_originales_necesarias].copy() # Usar .copy() para evitar SettingWithCopyWarning

        # Renombrar usando el mapeo actualizado
        df_renombrado = df_seleccionado.rename(columns=mapeo_actual)

        # Asegurar que todas las columnas objetivo existan, rellenando con NaN si es necesario
        for col_objetivo in nombres_objetivo.values():
            if col_objetivo not in df_renombrado.columns:
                df_renombrado[col_objetivo] = np.nan
                print(f"Info en {nombre}: Columna objetivo '{col_objetivo}' no generada por mapeo, creada con NaNs.")

        # Seleccionar el orden final de columnas basado en nombres_objetivo
        columnas_finales = [col for col in nombres_objetivo.values() if col in df_renombrado.columns]
        dfs_estandarizados[nombre] = df_renombrado[columnas_finales]

        print(f"DataFrame '{nombre}' estandarizado.")
    else:
        print(f"Advertencia: No se definió mapeo para '{nombre}'. Se omitirá.")



DataFrame 'Ordenes' estandarizado.
DataFrame 'Incidentes' estandarizado.
DataFrame 'Cambios' estandarizado.


In [ ]:
# Celda 8: Inspeccionar DataFrames Estandarizados
for nombre, df in dfs_estandarizados.items():
    print(f"\n--- {nombre} Estandarizado ---")
    print(df.head(3))
    print(df.info())
    print("-" * 30)


--- Ordenes Estandarizado ---
  ID_Original                                  Descripcion_Tarea  \
0     2124443                      MDL | reinicio nodo PSFTPRODM   
1     2121791                          MDL | Bajo | Generar dmps   
2     2080167  MDL | Alta | Reinicio nodos en warning PSFTPRO...   

                       Tecnico               Grupo Tipo_Ticket_Original  \
0         GORETTI ROMERO OCHOA  IT - Operproc-Apps     Orden de Trabajo   
1  JESUS AGUSTIN MARTINEZ CRUZ  IT - Operproc-Apps     Orden de Trabajo   
2  JESUS AGUSTIN MARTINEZ CRUZ  IT - Operproc-Apps     Orden de Trabajo   

  Origen_Proceso  
0        Ordenes  
1        Ordenes  
2        Ordenes  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5744 entries, 0 to 5743
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID_Original           5744 non-null   object
 1   Descripcion_Tarea     5744 non-null   object
 2   Tecnico 

In [ ]:
# Celda 9: Concatenar DataFrames
if dfs_estandarizados: # Verificar que haya DataFrames para concatenar
    df_unificado = pd.concat(dfs_estandarizados.values(), ignore_index=True)
    print("DataFrames consolidados en df_unificado.")
    print("\n--- Información del DataFrame Unificado ---")
    df_unificado.info()
    print("\n--- Primeras filas del DataFrame Unificado ---")
    print(df_unificado.head())
    print("\n--- Últimas filas del DataFrame Unificado ---")
    print(df_unificado.tail())
    print(f"\nNúmero total de registros: {len(df_unificado)}")
else:
    print("ERROR: No hay DataFrames estandarizados para consolidar.")
    df_unificado = pd.DataFrame() # Crear un DF vacío para evitar errores posteriores

DataFrames consolidados en df_unificado.

--- Información del DataFrame Unificado ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7575 entries, 0 to 7574
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID_Original           7575 non-null   object
 1   Descripcion_Tarea     7575 non-null   object
 2   Tecnico               7575 non-null   object
 3   Grupo                 7575 non-null   object
 4   Tipo_Ticket_Original  7575 non-null   object
 5   Origen_Proceso        7575 non-null   object
dtypes: object(6)
memory usage: 355.2+ KB

--- Primeras filas del DataFrame Unificado ---
  ID_Original                                  Descripcion_Tarea  \
0     2124443                      MDL | reinicio nodo PSFTPRODM   
1     2121791                          MDL | Bajo | Generar dmps   
2     2080167  MDL | Alta | Reinicio nodos en warning PSFTPRO...   
3     2084265                      RE: Reinici

In [ ]:
# Celda 10: Limpieza Final
if not df_unificado.empty:
    # Limpiar columna de descripción de tarea
    # Rellenar NaNs con string vacío ANTES de aplicar métodos de string
    df_unificado['Descripcion_Tarea'] = df_unificado['Descripcion_Tarea'].fillna('').astype(str)
    df_unificado['Descripcion_Tarea'] = df_unificado['Descripcion_Tarea'].str.lower().str.strip()

    # Eliminar filas donde la descripción de la tarea esté vacía después de limpiar
    original_rows = len(df_unificado)
    df_unificado = df_unificado[df_unificado['Descripcion_Tarea'] != '']
    print(f"Eliminadas {original_rows - len(df_unificado)} filas con Descripción de Tarea vacía.")

    # Eliminar duplicados (considera qué columnas definen una entrada única)
    # Podría ser ID_Original y Origen_Proceso, o incluir Descripcion_Tarea si buscas duplicados exactos
    subset_duplicates = ['ID_Original', 'Origen_Proceso', 'Descripcion_Tarea']
    # Asegurar que las columnas existen antes de usarlas en drop_duplicates
    subset_duplicates = [col for col in subset_duplicates if col in df_unificado.columns]

    if subset_duplicates:
        original_rows = len(df_unificado)
        df_unificado = df_unificado.drop_duplicates(subset=subset_duplicates)
        print(f"Eliminados {original_rows - len(df_unificado)} duplicados basados en {subset_duplicates}.")
    else:
        print("Advertencia: No se pudieron encontrar columnas clave para eliminar duplicados.")

    print("\n--- DataFrame Unificado Limpio (Primeras filas) ---")
    print(df_unificado.head())
    print(f"\nNúmero final de registros: {len(df_unificado)}")
else:
    print("DataFrame unificado está vacío, omitiendo limpieza.")

Eliminadas 0 filas con Descripción de Tarea vacía.
Eliminados 0 duplicados basados en ['ID_Original', 'Origen_Proceso', 'Descripcion_Tarea'].

--- DataFrame Unificado Limpio (Primeras filas) ---
  ID_Original                                  Descripcion_Tarea  \
0     2124443                      mdl | reinicio nodo psftprodm   
1     2121791                          mdl | bajo | generar dmps   
2     2080167  mdl | alta | reinicio nodos en warning psftpro...   
3     2084265                      re: reinicio nodos en warning   
4     2084041  mdl | bajo | reinicio de los 4 dominios de pla...   

                       Tecnico               Grupo Tipo_Ticket_Original  \
0         GORETTI ROMERO OCHOA  IT - Operproc-Apps     Orden de Trabajo   
1  JESUS AGUSTIN MARTINEZ CRUZ  IT - Operproc-Apps     Orden de Trabajo   
2  JESUS AGUSTIN MARTINEZ CRUZ  IT - Operproc-Apps     Orden de Trabajo   
3  JESUS AGUSTIN MARTINEZ CRUZ  IT - Operproc-Apps     Orden de Trabajo   
4  JESUS AGUSTIN MART

In [ ]:
# Celda 11: Análisis de Frecuencia Global
if not df_unificado.empty and 'Descripcion_Tarea' in df_unificado.columns:
    print("\n--- ANÁLISIS DE FRECUENCIA GLOBAL ---")
    task_frequency_global = df_unificado['Descripcion_Tarea'].value_counts().reset_index()
    task_frequency_global.columns = ['Descripcion_Tarea', 'Frecuencia_Global']

    # Mostrar las tareas más repetitivas (ej. Frecuencia > 1 o Top N)
    repetitive_tasks_global = task_frequency_global[task_frequency_global['Frecuencia_Global'] > 1].sort_values(by='Frecuencia_Global', ascending=False)
    print("\nTop 20 Tareas Repetitivas Globales (Frecuencia > 1):")
    print(repetitive_tasks_global.head(20).to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("No se puede realizar análisis de frecuencia global (DataFrame vacío o falta 'Descripcion_Tarea').")



--- ANÁLISIS DE FRECUENCIA GLOBAL ---

Top 20 Tareas Repetitivas Globales (Frecuencia > 1):
| Descripcion_Tarea                                                                                                        | Frecuencia_Global   |
|:-------------------------------------------------------------------------------------------------------------------------|:--------------------|
| procesar | reporte ambiente weblogic psftprodm                                                                           | 271                 |
| procesar_alertamiento_procesar dbf: se notifica el siguiente evento sobre la base de datos                               | 254                 |
| procesar_alertamiento_aie: f5_asm evento_critico_alertado                                                                | 216                 |
| equipos duplicados sentinel one                                                                                          | 141                 |
| procesar_alertamiento_a

In [ ]:

# Celda 12: Análisis de Frecuencia por Proceso
if not df_unificado.empty and 'Descripcion_Tarea' in df_unificado.columns and 'Origen_Proceso' in df_unificado.columns:
    print("\n--- ANÁLISIS DE FRECUENCIA POR PROCESO ---")
    task_frequency_proceso = df_unificado.groupby('Origen_Proceso')['Descripcion_Tarea'].value_counts().rename('Frecuencia').reset_index()

    # Mostrar las Top N tareas más frecuentes para cada proceso
    n_top = 10
    for proceso in df_unificado['Origen_Proceso'].unique():
        print(f"\nTop {n_top} Tareas Repetitivas para: {proceso}")
        top_tasks = task_frequency_proceso[task_frequency_proceso['Origen_Proceso'] == proceso].sort_values(by='Frecuencia', ascending=False).head(n_top)
        print(top_tasks.to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("No se puede realizar análisis por proceso (DataFrame vacío o faltan columnas).")




--- ANÁLISIS DE FRECUENCIA POR PROCESO ---

Top 10 Tareas Repetitivas para: Ordenes
| Origen_Proceso   | Descripcion_Tarea                                                                          | Frecuencia   |
|:-----------------|:-------------------------------------------------------------------------------------------|:-------------|
| Ordenes          | procesar | reporte ambiente weblogic psftprodm                                             | 271          |
| Ordenes          | procesar_alertamiento_procesar dbf: se notifica el siguiente evento sobre la base de datos | 254          |
| Ordenes          | procesar_alertamiento_aie: f5_asm evento_critico_alertado                                  | 216          |
| Ordenes          | equipos duplicados sentinel one                                                            | 141          |
| Ordenes          | procesar_alertamiento_aie: discovery:.procesar_firmas_ips_altas-criticas-no_bloqueadas     | 140          |
| Ordenes   

In [ ]:

# Celda 13: Identificar Tareas Comunes entre Procesos
if not df_unificado.empty and 'Descripcion_Tarea' in df_unificado.columns and 'Origen_Proceso' in df_unificado.columns:
    print("\n--- ANÁLISIS DE TAREAS COMUNES ENTRE PROCESOS ---")
    # Contar en cuántos procesos distintos aparece cada tarea
    task_process_count = df_unificado.groupby('Descripcion_Tarea')['Origen_Proceso'].nunique().reset_index()
    task_process_count.columns = ['Descripcion_Tarea', 'Num_Procesos_Distintos']

    # Filtrar tareas que aparecen en más de un proceso
    common_tasks = task_process_count[task_process_count['Num_Procesos_Distintos'] > 1].sort_values(by='Num_Procesos_Distintos', ascending=False)

    if not common_tasks.empty:
        print("\nTareas que aparecen en MÁS DE UN proceso:")
        # Podemos unir con la frecuencia global para más contexto
        common_tasks_detailed = pd.merge(common_tasks, task_frequency_global, on='Descripcion_Tarea', how='left')
        common_tasks_detailed = common_tasks_detailed.sort_values(by=['Num_Procesos_Distintos', 'Frecuencia_Global'], ascending=[False, False])
        print(common_tasks_detailed.head(20).to_markdown(index=False, numalign="left", stralign="left"))

        # Opcional: Ver exactamente en qué procesos aparece cada tarea común
        # print("\nDetalle de procesos para tareas comunes:")
        # common_tasks_list = common_tasks['Descripcion_Tarea'].tolist()
        # detail_common = df_unificado[df_unificado['Descripcion_Tarea'].isin(common_tasks_list)]\
        #     .groupby('Descripcion_Tarea')['Origen_Proceso'].unique().reset_index()
        # print(detail_common.head(20).to_markdown(index=False, numalign="left", stralign="left"))

    else:
        print("No se encontraron tareas comunes que aparezcan en más de un proceso.")
else:
    print("No se puede realizar análisis de tareas comunes (DataFrame vacío o faltan columnas).")


--- ANÁLISIS DE TAREAS COMUNES ENTRE PROCESOS ---

Tareas que aparecen en MÁS DE UN proceso:
| Descripcion_Tarea                                                                                                                                                                               | Num_Procesos_Distintos   | Frecuencia_Global   |
|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------|:--------------------|
| clon plantilla rh8 || procesar                                                                                                                                                                  | 2                        | 11                  |
| re: [tck#01208287] alta de servidor en previos poc f5                                                                                                                    

In [ ]:
# Celda 14: Definir Función de Categorización (Mejorada Ligeramente)
def categorize_task(task):
    # Asegurarse que task sea string y manejar posibles None/NaN residuales
    if not isinstance(task, str):
        return "Categoría Desconocida"
    task = task.lower() # Ya debería estar en minúsculas, pero por seguridad

    # Ordenar de más específico a más general o por prioridad
    if any(keyword in task for keyword in ["batch", "control-m", "query", "shell", "script", "proceso nocturno"]):
        return "Procesamiento Batch y Scripts"
    elif any(keyword in task for keyword in ["respaldo", "backup", "restauración", "recovery"]):
        return "Respaldo y Recuperación"
    elif any(keyword in task for keyword in ["reporte", "monitoreo", "alertamiento", "dashboard", "informe", "health check"]):
        return "Monitoreo y Reporte"
    elif any(keyword in task for keyword in ["mantenimiento", "actualización", "parche", "upgrade", "update", "preventivo"]):
        return "Mantenimiento y Actualización"
    elif any(keyword in task for keyword in ["depura", "limpieza", "optimización", "tuning", "espacio en disco"]):
        return "Limpieza y Optimización"
    elif any(keyword in task for keyword in ["acceso", "permiso", "usuario", "contraseña", "password", "rol", "alta", "baja"]):
        return "Gestión de Accesos y Usuarios"
    elif any(keyword in task for keyword in ["servidor", "vm", "instancia", "nodo", "cluster", "storage", "red", "firewall"]):
        return "Gestión de Infraestructura (Servidores, Red, Storage)"
    elif any(keyword in task for keyword in ["cliente", "atención", "solicitud", "requerimiento", "soporte", "duda"]):
        return "Soporte y Atención a Solicitudes"
    elif any(keyword in task for keyword in ["equipo", "agente", "instalación", "configuración", "despliegue", "deploy"]):
        return "Instalación y Configuración"
    elif task == "": # Capturar tareas que quedaron vacías
         return "Descripción Vacía"
    else:
        return "Otras Tareas / Sin Categorizar"


In [ ]:
# Celda 15: Aplicar Categorización y Analizar
if not df_unificado.empty and 'Descripcion_Tarea' in df_unificado.columns:
    df_unificado['Naturaleza_Tarea'] = df_unificado['Descripcion_Tarea'].apply(categorize_task)

    print("\n--- ANÁLISIS POR CATEGORÍA DE TAREA ---")
    print("\nDistribución Global de Categorías:")
    print(df_unificado['Naturaleza_Tarea'].value_counts().to_markdown(numalign="left", stralign="left"))

    if 'Origen_Proceso' in df_unificado.columns:
      print("\nDistribución de Categorías por Proceso:")
      category_by_process = df_unificado.groupby('Origen_Proceso')['Naturaleza_Tarea'].value_counts(normalize=True).mul(100).round(2).unstack(fill_value=0)
      # Usar .unstack() para tener procesos como columnas y categorías como filas (o viceversa)
      # category_by_process = df_unificado.groupby(['Origen_Proceso', 'Naturaleza_Tarea']).size().unstack(fill_value=0)
      print(category_by_process.to_markdown(numalign="left", stralign="left"))

    print("\n--- DataFrame Final con Categorías (Primeras filas) ---")
    print(df_unificado.head())

else:
    print("No se puede aplicar categorización (DataFrame vacío o falta 'Descripcion_Tarea').")


--- ANÁLISIS POR CATEGORÍA DE TAREA ---

Distribución Global de Categorías:
| Naturaleza_Tarea                                      | count   |
|:------------------------------------------------------|:--------|
| Otras Tareas / Sin Categorizar                        | 3611    |
| Monitoreo y Reporte                                   | 1942    |
| Gestión de Accesos y Usuarios                         | 711     |
| Procesamiento Batch y Scripts                         | 399     |
| Gestión de Infraestructura (Servidores, Red, Storage) | 289     |
| Instalación y Configuración                           | 220     |
| Mantenimiento y Actualización                         | 201     |
| Respaldo y Recuperación                               | 110     |
| Soporte y Atención a Solicitudes                      | 88      |
| Limpieza y Optimización                               | 4       |

Distribución de Categorías por Proceso:
| Origen_Proceso   | Gestión de Accesos y Usuarios   | Gestión de 